# Hilal durmaz
**Approach2:** Item to item collaborative filtering method.

In [1]:
import pandas as pd
import numpy as np
from sklearn import neighbors as nei
from sklearn import metrics as met

In [2]:
data = pd.read_csv("train.csv")
data.head()

,userID,movieID,rating
0,4490,2109,4
1,5839,3471,4
2,5382,150,3
3,1262,1237,5
4,6005,2273,4


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990209 entries, 0 to 990208
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   userID   990209 non-null  int64
 1   movieID  990209 non-null  int64
 2   rating   990209 non-null  int64
dtypes: int64(3)
memory usage: 22.7 MB


In [4]:
data.shape

(990209, 3)

In [6]:
U_user = data.pivot("movieID","userID", "rating")
U_user

userID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,6001,6002,6003,6004,6005,6006,6007,6008,6009,6010,6011,6012,6013,6014,6015,6016,6017,6018,6019,6020,6021,6022,6023,6024,6025,6026,6027,6028,6029,6030,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movieID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,3.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,3.0,NaN,NaN,NaN,NaN,4.0,5.0,NaN,5.0,NaN,NaN,5.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,5.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3.0,NaN,NaN,4.0,NaN,3.0,NaN,5.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,4.0,NaN,4.0,NaN,NaN,NaN,NaN,5.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,3.0,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#normalization
User_scaled = U_user - U_user.mean(axis=1).values.reshape(-1, 1)
User_scaled.fillna(0, inplace=True)
User_scaled.head()

userID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,6001,6002,6003,6004,6005,6006,6007,6008,6009,6010,6011,6012,6013,6014,6015,6016,6017,6018,6019,6020,6021,6022,6023,6024,6025,6026,6027,6028,6029,6030,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movieID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,-1.145287,0.0,0.0,0.0,0.0,-0.145287,0.0,0.0,-0.145287,0.0,0.0,0.0,0.0,0.0,0.0,0.854713,0.0,0.0,0.854713,-1.145287,0.0,0.0,0.0,0.0,-0.145287,0.854713,0.0,0.854713,0.0,0.0,0.854713,0.0,0.0,0.0,-2.145287,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.854713,0.0,0.854713,0.0,0.854713,0.0,0.0,0.0,0.0,0.0,-1.145287,0.0,-1.145287,0.0,0.0,-0.145287,0.0,-1.145287,0.0,0.854713,-0.145287,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.854713,0.854713,-0.145287,0.0,-0.145287,0.0,0.0,0.0,0.0,0.854713
2,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.8,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.8,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,-1.200000,0.0,0.0,0.0,0.0,-0.2,...,1.8,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,-2.2,0.000000,0.0,0.0,-1.200000,0.0,0.000000,0.0,0.000000,-1.200000,0.0,0.0,0.0,0.0,-0.2,0.0,0.0,1.800000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
3,0.000000,0.0,0.0,0.0,0.0,-2.021142,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,-0.021142,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,-1.021142,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,-1.021142,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
4,0.000000,0.0,0.0,0.0,0.0,-0.733728,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.266272,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.266272,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.266272,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
5,0.000000,0.0,0.0,0.0,0.0,-2.017007,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.982993,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000


In [8]:
#train test split
train_size = 500
train_u = User_scaled[:train_size]
test_u = User_scaled[train_size:]

# 1.Structure

In [ ]:
nn = nei.NearestNeighbors(n_neighbors=train_size, leaf_size=30, metric='cosine')
nn.fit(train_u)


NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=500, p=2,
                 radius=1.0)

In [ ]:
myPredsu = []
realLabelsu = []
k = 100
for movieId in test_u.index:
  print(movieId)
  if movieId > 520:
    break
  for userID in test_u.columns:
    
    if not np.isnan(U_user.loc[movieId,userID ]): 
      test_user = test_u.loc[movieId:movieId, :]
      indices = nn.kneighbors(test_user, return_distance=False)[0]+1 

      ratings = []
      i = 0
      while len(ratings) < k and i < len(indices):
        if not np.isnan(U_user.iloc[indices[i], userID]):
          ratings.append(U_user.iloc[indices[i], userID])
        i += 1

      if len(ratings) > 0:
        pred = sum(ratings)/len(ratings)
        myPredsu.append(pred)

        real_label = U_user.loc[movieId,userID]
        realLabelsu.append(real_label)

514
515
516
517
518
519
520
521


In [ ]:
met.mean_squared_error(realLabelsu, myPredsu, squared=False)

1.4264585060164185

# **2.Strategy**

In [ ]:
k = 100
myPreds_ıtem = []
realLabels_ıtem = []
for movieId in test_u.index:
  print(movieId)
  if movieId > 520:
    break
  for userID in test_u.columns:
    if not np.isnan(test_u.loc[movieId, userID]):
      filtered_train = train_u.dropna(subset=[userID])
      filtered_train = filtered_train - filtered_train.mean(axis=1).values.reshape(-1,1)
      filtered_train.fillna(0, inplace=True)
      if k < len(filtered_train):
        nn = nei.NearestNeighbors(n_neighbors=k,metric="correlation")
        nn.fit(filtered_train)

        test_row = test_u.loc[movieId:movieId, :]
        test_row = test_row - test_row.mean(axis=1).values.reshape(-1,1)
        test_row.fillna(0, inplace=True)

        indices = nn.kneighbors(test_row, return_distance=False)[0]

        userIds = filtered_train.iloc[indices, :].index
        pred = train_u.loc[userIds, userID].values.mean()
        myPreds_ıtem.append(pred)

        real_label = test_u.loc[movieId,userID ]
        realLabels_ıtem.append(real_label)


514
515
516
517
518
519
520
521


In [ ]:
met.mean_squared_error(realLabels_ıtem,myPreds_ıtem, squared=False)

0.21286636445543142

# **Prediction**

In [9]:
 test = pd.read_csv("test_without_labels.csv")
test.head()

,IDs
0,629_2683
1,601_904
2,5673_3717
3,5616_1721
4,1099_3697


In [10]:
userid=[]
movieId=[]

x=list(test.IDs.values)
for i in range(test.shape[0]):
  user,movie=x[i].split("_")
  userid.append(user)
  movieId.append(movie)


d = {'UserId':userid , 'MovieId': movieId}
df_Test = pd.DataFrame(data=d)


In [11]:
df_Test.head()

,UserId,MovieId
0,629,2683
1,601,904
2,5673,3717
3,5616,1721
4,1099,3697


In [12]:
k = 10
myPreds = []

for i in range(len(df_Test)):
  userId=int(df_Test.UserId[i])
  movieId=int(df_Test.MovieId[i])
  filtered_train = U_user.dropna(subset=[userId])
  filtered_train = filtered_train - filtered_train.mean(axis=1).values.reshape(-1,1)
  filtered_train.fillna(0, inplace=True)
  if k < len(filtered_train):
    nn = nei.NearestNeighbors(n_neighbors=k,metric="correlation",)
    nn.fit(filtered_train)

    test_row = U_user.loc[movieId:movieId, :]
    test_row = test_row - test_row.mean(axis=1).values.reshape(-1,1)
    test_row.fillna(0, inplace=True)

    indices = nn.kneighbors(test_row, return_distance=False)[0]

    userIds = filtered_train.iloc[indices, :].index
    pred = U_user.loc[userIds, userId].values.mean()
    myPreds.append(pred)

  else:
    test_row2 =U_user.loc[movieId:movieId, :]
    pred2=test_row2.mean(axis=1).values.reshape(-1,1)[0][0]
    myPreds.append(pred2)


In [13]:
submission = pd.DataFrame({"IDs":list(test.IDs.values), "rating":myPreds})
submission.to_csv("submission_apc2.csv", index = None)
from google.colab import files

files.download('submission_apc2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>